In [ ]:
from google.colab import drive
drive.mount("/content/drive")
# drive.mount("/content/drive", force_remount=True)

In [ ]:
!pip install virtualenv

In [ ]:
!virtualenv /content/drive/MyDrive/colab_env

In [ ]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [ ]:
# Add the path of the virtual environment 'site-packages' to the colab system path
import sys
sys.path.append("/content/drive/MyDrive/colab_env/lib/python3.8/site-packages")

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow-text

In [ ]:
!pip install stanza

In [ ]:
def load_text(filename):
  '''# Load a text file.'''
  with open(filename, 'r', encoding='utf-8') as file:
    lines = file.readlines()
  
  texts = [line.replace('\\n', '\n')
           for line in lines
           if len(line.strip()) > 0]

  return texts

def load_data(directory):
  '''Load all text files in a directory.'''
  texts = []
  for filename in os.listdir(directory):
    if (filename.endswith('txt')):
      texts.extend(load_text(os.path.join(directory, filename)))
  return texts

def split_sentences(text):
    sentences = re.split('(?<=[.!?]) +', text)
    return sentences

In [ ]:
import os
import re

# Specify the directory where your texts are located
directory = '/content/drive/MyDrive/Colab Notebooks/corpus'
texts = load_data(directory)

# Split each text into sentences
split_texts = [split_sentences(text) for text in texts]

# Flatten the list of lists of sentences
sents = [sentence for text in split_texts for sentence in text]
print(sum([len(sent.split(' ')) for sent in sents]))

6375


In [ ]:
import stanza

# Download the Arabic models for Stanza
stanza.download('ar')

# Initialize the pipeline
nlp = stanza.Pipeline(lang = 'ar')

# Process each sentence
docs = [nlp(sentence) for sentence in sents]

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

TOTAL_VOCAB_SIZE = 2000

# Convert each document to a string
texts = [' '.join([word.lemma for sent in doc.sentences for word in sent.words]) for doc in docs]

tokenizer = Tokenizer(lower = False, oov_token = '<OOV>', num_words = TOTAL_VOCAB_SIZE)

# Fit the tokenizer on the texts
tokenizer.fit_on_texts(texts)

# Convert the texts to sequences of word indices
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
# The tokenizer's word_index attribute is a dictionary mapping words to their index
# {'word': 'index'}, 'index' being the word frequency, most frequent ones at the beginning

# 'word': 'index'
word_index_dict = tokenizer.word_index

# 'index': 'word
index_word_dict = {value: key for key, value in word_index_dict.items()}

print(word_index_dict.items())
print(index_word_dict.items())

dict_items([('<OOV>', 1), ('وَ', 2), ('هُوَ', 3), ('فِي', 4), ('،', 5), ('بِ', 6), ('لِ', 7), ('مِن', 8), ('فَ', 9), ('كَان', 10), ('عَلَى', 11), ('ت', 12), ('سَفِينَة', 13), ('إِلَى', 14), ('اَلَّذِي', 15), ('بَحر', 16), ('UNK', 17), ('أَنَّ', 18), ('صِنَاعَة', 19), ('‘', 20), ('؛', 21), ('إِنَّ', 22), ('بَحرِيّ', 23), ('مَا', 24), ('قَال', 25), ('مُرَاكِب', 26), ('ذٰلِكَ', 27), ('عَن', 28), ('مُسلِم', 29), ('هٰذَا', 30), ('أَسطِيل', 31), ('بِن', 32), ('أُسطُول', 33), ('حَربِيّ', 34), ('قَد', 35), ('بَعدَ', 36), ('ابن', 37), ('أُندُلس', 38), ('مِصر', 39), ('كَمَا', 40), ('لَا', 41), ('مَدِينَة', 42), ('حَتَّى', 43), ('إِسلَامِيّ', 44), ('مُتَوَسِّط', 45), ('قَرن', 46), ('جَزِيرَة', 47), ('عمر', 48), ('ذَكَر', 49), ('كُلّ', 50), ('مَعَ', 51), ('لَم', 52), ('سَاحِل', 53), ('جَمع', 54), ('دَار', 55), ('دَولَة', 56), ('أَلف', 57), ('حَمَل', 58), ('فَتح', 59), ('هـ', 60), ('إِ', 61), ('يَوم', 62), ('عُثمَانِيّ', 63), ('قُوَّة', 64), ('خَلِيفَة', 65), ('عبد', 66), ('أَمر', 67), ('إِنشَاء', 

In [ ]:
print(f'Documents: {len(docs)}')

print(f'\nTokens (sentences): {sum([len(sent.tokens) for doc in docs for sent in doc.sentences])}')
#print(sum([doc.num_tokens for doc in docs]))
print(f'Indices (sequences): {sum([len(seq) for seq in sequences])}')

# IMPORTANT: tokenizer.vocab_index saves all tokens,
# not just the TOTAL_VOCAB_SIZE most frequent ones
print(f'\nTotal vocab size: {len(word_index_dict)} {len(index_word_dict)}')

# BUT: TOTAL_VOCAB_SIZE taken into account when generating sequences: texts_to_sequences()
uniq_indices = np.unique(np.concatenate(np.array(sequences)))
print(f'Unique tokens in sequences: {uniq_indices.size}\n')

Documents: 163

Tokens (sentences): 8153
Indices (sequences): 8615

Total vocab size: 2505 2505
Unique tokens in sequences: 1999



<ipython-input-16-dd993308125b>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  uniq_indices = np.unique(np.concatenate(np.array(sequences)))


In [ ]:
for i, doc in enumerate(docs[:5]):
  sentence = [token.lemma for token in doc.sentences[0].words]
  print(f'{len(sentence)} - {sentence}')
  print(f'{len(sequences[i])} - {sequences[i]}')

6 - ['تَارِيخ', 'مُؤَسَّسَة', 'بَحرِيَّة', 'إِسلَامِيّ', '.', '.']
4 - [74, 225, 75, 44]
16 - ['حَاصَر', 'قِسطَنطِينِيَّة', '29', 'مَرَّة', 'وضمّ', 'أُسطُول', 'هُوَ', 'مِصرِيّ', '36', 'أَلف', 'سَفِينَة', 'وَ', 'وَضَع', 'أُدُق', 'خَرِيطَة', 'لأميركا']
16 - [798, 226, 227, 444, 799, 33, 3, 91, 445, 57, 13, 2, 228, 800, 92, 801]
3 - ['محمد', 'شعبان', 'أيوب']
3 - [229, 446, 802]
57 - ['قَال', 'مُؤَرَّخ', 'أَمِيرِكِيّ', 'ويل', 'ديورانت', '(', 'ت', '1402هـ', '/', '1981م', ')', '-', 'فِي', '’قصة', 'حَضَارَة', '’', '-', 'إِنَّ', 'هُوَ', 'كَان', 'لِ', 'مُسلِم', 'بِ', 'أُندُلس', '"', 'أُسطُول', 'تِجَارِيّ', 'زَاد', 'عَلَى', 'أَلف', 'سَفِينَة', 'حَمَل', 'غُلّ', 'أُندُلس', 'وَ', 'مَصنُوع', 'هُوَ', 'إِلَى', 'أَفرِيقِيّ', 'وَ', 'آسِي', '،', 'وَ', 'كَان', 'سَفِينَة', 'قَادِم', 'مِن', 'مِئَة', 'ثَغر', 'وَ', 'ثَغر', 'اِزدَحَم', 'بِ', 'هُوَ', '"', '!', '!']
48 - [25, 447, 171, 803, 448, 12, 804, 805, 4, 449, 172, 230, 22, 3, 10, 7, 29, 6, 38, 33, 231, 303, 11, 57, 13, 58, 450, 38, 2, 451, 3, 14, 141, 2,

In [ ]:
more = []
less = []

for i, doc in enumerate(docs):
  sentence = [token.lemma for token in doc.sentences[0].words]
  more.append(len(sentence) > len(sequences[i]))
  less.append(len(sentence) < len(sequences[i]))

print(f'Sentence len > sequence len: {more.count(True), less.count(True)}')
print(f'Sentence len < sequence len: {less.count(True), less.count(True)}')

Sentence len > sequence len: (146, 0)
Sentence len < sequence len: (0, 0)


In [ ]:
# Verify the word index, find OOV tokens

oov = []

for i, doc in enumerate(docs):
  sentence = [token.lemma for token in docs[i].sentences[0].words]
  for token in sentence:
    if tokenizer.word_index.get(token) is None:
      oov.append(token)

oov_list = sorted(set(oov))
print(len(oov_list))
print(oov_list)

62
['!', '"', '(', ')', '-', '.', '/', ':', '<UNK>', '<UNK>رجال<UNK>', '<UNK>ف<UNK>دل', '<UNK>فهي', '<UNK>من', '<UNK>والي', '=', '[بن', '[بيبرس', '[حجمُها]', '[حسان]', '[دار]', '[سفن]', '[غزو]', '[فـ]ـلا', '[فيها]', '[قائد', '[كانوا]', '[مختلف', '[نار]', ']', 'إ<UNK>تْمامك', 'إذ<UNK>', 'الأساطيل<UNK>', 'الأسطول]', 'الإنسان<UNK>', 'الاعتبار<UNK>', 'البرتغاليين]', 'البزنطيون<UNK>', 'البوارج<UNK>', 'الح<UNK>ذْق', 'الخبر<UNK>', 'الز<UNK>ففْت', 'السفن]', 'الشام<UNK>', 'العباسي]', 'العراق<UNK>', 'القار<UNK>', 'الممالك<UNK>', 'انتخخ<UNK>', 'بز<UNK>', 'بغداد]', 'ج<UNK>هازَ', 'حمالات<UNK>', 'عصرهـ[ـا]', 'مركبا<UNK>', 'مصر]', 'من]', 'منحد<UNK>رة', 'مَقَ<', 'ه<UNK>', 'يحْس<UNK>ن', 'يَر<UNK>د', 'ُجَج<UNK>']


In [ ]:
### COUNT TOKEN FREQUENCIES, EXTRACT THE MOST COMMON WORDS

from collections import Counter

# Flatten the list of sequences [list of lists of token ids] -> [list of token ids]
flat_sequences = [token_id for sequence in sequences for token_id in sequence]

# Count the frequency of each token_id
token_frequencies = Counter(flat_sequences)

# Get the 10 most common token_ids ('token_id', 'frequency')
most_common_dict = dict(token_frequencies.most_common(TOTAL_VOCAB_SIZE))

In [ ]:
print(f'Total # token ids: {len(flat_sequences)}')
print(f'# tokens: {len(token_frequencies)}')
print(f'# most common tokens: {len(most_common_dict)}' )

Total # token ids: 8615
# tokens: 1999
# most common tokens: 1999


In [ ]:
### BUILDING THE NEXT TOKEN DATASET

def build_next_token_set(sequence, min_sequence_length = 3):
  data = [(sequence[:i], sequence[i]) for i in range(min_sequence_length, len(sequence)-1)]
  data.append((sequence[:len(sequence)-1], sequence[-1]))
  return data

def build_next_token_dataset(sequences, min_sequence_length = 3):
  labeled_dataset = []

  for sequence in sequences:
    data = build_next_token_set(sequence, min_sequence_length = min_sequence_length)
    labeled_dataset.extend(data)

  return labeled_dataset

In [ ]:
MIN_SEQUENCE_LENGTH = 3
labeled_dataset = build_next_token_dataset(sequences, min_sequence_length = MIN_SEQUENCE_LENGTH)

print(f'Sequences: {len(sequences)}')
print(f'Labeled dataset: {len(labeled_dataset)}')

for sequence, label in labeled_dataset[:10]:
    print(f"sequence: {sequence}, label: {label}")

Sequences: 163
Labeled dataset: 8159
sequence: [74, 225, 75], label: 44
sequence: [798, 226, 227], label: 444
sequence: [798, 226, 227, 444], label: 799
sequence: [798, 226, 227, 444, 799], label: 33
sequence: [798, 226, 227, 444, 799, 33], label: 3
sequence: [798, 226, 227, 444, 799, 33, 3], label: 91
sequence: [798, 226, 227, 444, 799, 33, 3, 91], label: 445
sequence: [798, 226, 227, 444, 799, 33, 3, 91, 445], label: 57
sequence: [798, 226, 227, 444, 799, 33, 3, 91, 445, 57], label: 13
sequence: [798, 226, 227, 444, 799, 33, 3, 91, 445, 57, 13], label: 2


In [ ]:
### PADDING THE SEQUENCES

from tensorflow.keras.preprocessing.sequence import pad_sequences

SEQUENCE_PADDING_LENGTH = 20

data_sequences, data_labels = zip(*labeled_dataset)

padded_sequences = pad_sequences(data_sequences, maxlen = SEQUENCE_PADDING_LENGTH, padding = 'post')
print(data_sequences[100])
print(padded_sequences[100])

[457, 810, 82, 7, 29, 15, 811, 120, 13, 812, 8, 813, 2]
[457 810  82   7  29  15 811 120  13 812   8 813   2   0   0   0   0   0
   0   0]


In [ ]:
#### BUILDING THE MODEL

from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

# Building the model
model = Sequential([
    Embedding(input_dim = TOTAL_VOCAB_SIZE + 1, output_dim = 100),
    LSTM(128),
    Dense(TOTAL_VOCAB_SIZE + 1, activation='softmax')
])

# Compiling the model
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])

# Converting labels to numpy array as it is expected by the model
data_labels = np.array(data_labels)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data_labels, test_size=0.2)

# Training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
204/204 [==============================] - 17s 72ms/step - loss: 6.3273 - accuracy: 0.0729 - val_loss: 6.1133 - val_accuracy: 0.0833
Epoch 2/10
204/204 [==============================] - 3s 16ms/step - loss: 5.7982 - accuracy: 0.0852 - val_loss: 6.2087 - val_accuracy: 0.0741
Epoch 3/10
204/204 [==============================] - 2s 11ms/step - loss: 5.7002 - accuracy: 0.0954 - val_loss: 6.1983 - val_accuracy: 0.1146
Epoch 4/10
204/204 [==============================] - 2s 11ms/step - loss: 5.5603 - accuracy: 0.1233 - val_loss: 6.1952 - val_accuracy: 0.1268
Epoch 5/10
204/204 [==============================] - 2s 8ms/step - loss: 5.4445 - accuracy: 0.1382 - val_loss: 6.3020 - val_accuracy: 0.1348
Epoch 6/10
204/204 [==============================] - 2s 10ms/step - loss: 5.3140 - accuracy: 0.1491 - val_loss: 6.3439 - val_accuracy: 0.1366
Epoch 7/10
204/204 [==============================] - 1s 6ms/step - loss: 5.2280 - accuracy: 0.1552 - val_loss: 6.4145 - val_accuracy: 0.1373


In [ ]:
### EVALUATE THE MODEL

loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test accuracy:", accuracy)

51/51 - 0s - loss: 6.5532 - accuracy: 0.1397 - 137ms/epoch - 3ms/step
Test accuracy: 0.13970588147640228


In [ ]:
# https://www.kaggle.com/datasets/haithemhermessi/sanad-dataset?resource=download